In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import re

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\j5081\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


https://edition.cnn.com/2021/10/07/health/boosters-outpace-initial-vaccines/index.html

In [19]:
max_day = dt.datetime.strptime("2020-3-31", "%Y-%m-%d")
min_day = dt.datetime.strptime("2020-3-1", "%Y-%m-%d")

title_list_list=[]
date_list=[]
body_list=[]

while(1):

    # 검색 날짜 지정
    if (min_day.month in [1,3,5,7,8,10,12]):
        delta = relativedelta(days=31)
    elif (min_day.month in [4,6,9,11]):
        delta = relativedelta(days=30)
    else:
        delta = relativedelta(days=28)
    
    max_day = max_day - delta
    min_day = max_day
    min_day = max_day.replace(day=1)

    if (min_day.month not in [2]):
        break


    # 검색 url 지정
    url_start = "https://www.google.com/search?q=covid+19+cnn+~vaccine+OR+~vaccination&biw=1920&bih=924&source=lnt&tbs=cdr%3A1%2Ccd_min%3A"
    
    
    url_min_month = str(min_day.month)
    url_min_day = str(min_day.day)
    url_min_year = str(min_day.year)

    url_max_start = "%2Ccd_max%3A"
    url_max_month = str(max_day.month)
    url_max_day = str(max_day.day)
    url_max_year = str(max_day.year)

    url_end = "&tbm=nws"

    between = "%2F"

    url = url_start + url_min_month + between + url_min_day + between + url_min_year + url_max_start + url_max_month + between + url_max_day + between + url_max_year + url_end
    print(url)

    # 해당 날짜의 url에 접속
    driver.get(url)
    #print("get")

    html=driver.page_source
    #print("html1:",html[:100])
    soup=BeautifulSoup(html, 'html.parser')

    is_cnn=soup.select("div.CEMjEf > span")#rso > div:nth-child(3) > g-card > div > div > a > div > div.iRPxbe > div.CEMjEf > span
    print("get is cnn")

        
    i=0
    page_num=3
    title_list=[]
    
    while(len(title_list)<50):
        print(len(title_list))
        html=driver.page_source
        soup=BeautifulSoup(html, 'html.parser')
        

        i=i+1
        
        

        if i%8==0:
            html=driver.page_source
            soup=BeautifulSoup(html, 'html.parser')

            page_num_selector = "#xjs > table > tbody > tr > td:nth-child(" + str(page_num) + ") > a > span"
            driver.find_element_by_css_selector(page_num_selector).click()
            page_num = page_num + 1
            i=1

        print(i-1)

        is_cnn=soup.select("div.CEMjEf > span")
        #print("len is cnn:", len(is_cnn))
        print("is_cnn",is_cnn[i-1].string)
        
        if is_cnn[i-1].string == "CNN":
            #print("CNN" == is_cnn[i-1].string)
            #try:        
            news = "#rso > div:nth-child(" + str(i) + ") > g-card > div > div > a > div > div.iRPxbe > div.mCBkyc.JQe2Ld.nDgy9d"
            
            driver.find_element_by_css_selector(news).click()
            print("클릭")
            
            html=driver.page_source
            soup=BeautifulSoup(html, 'html.parser')

            try:
                # 날짜
                date=soup.select("p.update-time")

                date_re = re.search("\) ", date[0].text)
                date_1=date[0].text[date_re.end():-1]
                date_2=dt.datetime.strptime(date_1, '%B %d, %Y')
                date_clean = str(date_2.year)+"-"+str(date_2.month)+"-"+str(date_2.day)
                print(date_clean)
                
                
                # 제목
                title=soup.select("h1")
                
                #title_list.append(title[0].text)

                print("------",title[0].text)


                # 본문1
                body_str=""
                body_1st = soup.select("#body-text > div.l-container > div.el__leafmedia.el__leafmedia--sourced-paragraph")

                body_re = re.search("(CNN)", body_1st[0].text)
                body_clean = body_1st[0].text[body_re.end()+1:]

                if body_clean != "":
                    body_str = body_str + body_clean

                    # 본문2
                body_2nd = soup.find_all("div", class_="zn-body__paragraph")
                for paragraph_num in range(len(body_2nd)):
                    #print(body_2nd[paragraph_num].text)

                    if body_2nd[paragraph_num].text not in  ["", " "]:
                        body_str = body_str + body_2nd[paragraph_num].text
                
                title_list.append(title[0].text)
                date_list.append(date_clean)
                body_list.append(body_str.replace("  "," "))
                #print(body_list)
            

                driver.back()

            except:
                driver.back()

            html=driver.page_source
            #print("html2:",html[:100])
            soup=BeautifulSoup(html, 'html.parser')
            is_cnn=soup.select("div.CEMjEf > span")
    for list_num in range(len(title_list)):
        title_list_list.append(title_list[list_num])



https://www.google.com/search?q=covid+19+cnn+~vaccine+OR+~vaccination&biw=1920&bih=924&source=lnt&tbs=cdr%3A1%2Ccd_min%3A2%2F1%2F2020%2Ccd_max%3A2%2F29%2F2020&tbm=nws
get is cnn
0
0
is_cnn CNN
클릭
2020-2-26
------ Biotech company Moderna says its coronavirus vaccine is ready for first tests 
1
1
is_cnn CNN
클릭
2020-3-1
------ First death from coronavirus in the United States confirmed in Washington state
2
2
is_cnn CNN
클릭
2020-2-22
------ A controversial religious group is at the center of South Korea's coronavirus outbreak
3
3
is_cnn CNN
클릭
2020-2-19
------ Chinese CDC study finds Covid-19 virus to be more contagious than SARS or MERS
4
4
is_cnn CNN
클릭
2020-2-26
------ Coronavirus has now spread to every continent except Antarctica
5
5
is_cnn CNN
클릭
2020-2-26
------ CDC official warns Americans it's not a question of if coronavirus will spread, but when
6
6
is_cnn CNN
클릭
2020-2-21
------ China's changed how it counts virus cases three times now. Here's why
7
0
is_cnn CNN
클릭
2020-2-14
--

IndexError: list index out of range

In [50]:
date_list[45:]


['2020-2-8', '2020-2-3', '2020-2-19', '2020-2-15', '2020-2-11']

In [55]:
len(title_list_list)

50

In [56]:
dict_ = {}
dict_["date"] = date_list
dict_["title"] = title_list_list
dict_["text"] = body_list
df = pd.DataFrame(dict_)
df

,date,title,text
0,2020-2-26,Biotech company Moderna says its coronavirus v...,Business)US biotech firm Moderna has shipped a...
1,2020-3-1,First death from coronavirus in the United Sta...,A patient infected with the novel coronavirus ...
2,2020-2-22,A controversial religious group is at the cent...,More than half of South Korea's novel coronavi...
3,2020-2-19,Chinese CDC study finds Covid-19 virus to be m...,"A comprehensive study of more than 72,000 conf..."
4,2020-2-26,Coronavirus has now spread to every continent ...,Public health officials warned Wednesday that ...
5,2020-2-26,CDC official warns Americans it's not a questi...,One of the top officials at the Centers for Di...
6,2020-2-21,China's changed how it counts virus cases thre...,Weeks after the novel coronavirus crisis began...
7,2020-2-14,"Over 1,700 frontline medics infected with coro...","Ning Zhu, a nurse in Wuhan, the central Chines..."
8,2020-2-17,Did Xi Jinping know about the coronavirus outb...,As the deadly novel coronavirus spread through...
9,2020-2-13,Some coronavirus test kits shipped to states a...,Some of the coronavirus test kits shipped to l...


In [17]:
df.to_csv("20_2.csv")

In [7]:
pd.read_csv("21_9,10.csv")

,Unnamed: 0,date,title,text
0,0,2021-10-15,FDA vaccine advisers recommend emergency use a...,Vaccine advisers to the US Food and Drug Admin...
1,1,2021-10-8,Studies confirm waning immunity from Pfizer's ...,Two real-world studies published Wednesday con...
2,2,2021-10-8,Here's what having a Covid-19 vaccine for chil...,Pfizer said Thursday it's asked the US Food an...
3,3,2021-10-12,Texas governor bans Covid-19 vaccine mandates ...,Texas Gov. Greg Abbott on Monday issued an exe...
4,4,2021-10-4,NYC vaccine mandate takes effect with 96% of t...,New York City Mayor Bill de Blasio said 96% of...
...,...,...,...,...
95,95,2021-9-14,Russia's Vladimir Putin is quarantining after ...,Russian President Vladimir Putin is quarantini...
96,96,2021-9-20,Cuba vaccinates children as young as 2 in stra...,The Cuban children sat in a Havana hospital wa...
97,97,2021-9-29,California's second-largest school district vo...,The board of California's second-largest schoo...
98,98,2021-9-20,"Nearly 226,000 Covid-19 cases were reported in...",Covid-19 infections have continued to increase...
